<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Belén Ordenes
- Nombre de alumno 2: Javiera Donoso


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/javieradonoso/Repositorio-MDS7202)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
#!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer, KNNImputer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
df = pd.read_pickle('online_retail_data.pickle')
df

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [ ]:
def explore_data(dataframe_in):
    # histograma para las variables precios y cantidad
    fig_price = px.histogram(dataframe_in, x='Price', title='Histograma de Precios')
    fig_quantity = px.histogram(dataframe_in, x='Quantity', title='Histograma de Cantidad')
    fig_price.show()
    fig_quantity.show()

    # Imprime datos nulos por variable
    null_counts = dataframe_in.isnull().sum()
    print("Conteo de datos nulos por variable:")
    print(null_counts)
    pass

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [ ]:
class IQR(BaseEstimator, TransformerMixin):
    def __init__(self, lambd):
        # Definir el parámetro lambda
        self.lambd = lambd

    def fit(self, X, y=None):
        # Calcular Q1 y Q3 
        self.Q1 = X.quantile(0.25)
        self.Q3 = X.quantile(0.75)
        # Calcular el rango intercuartil
        self.IQR = self.Q3 - self.Q1
        return self

    def transform(self, X, y=None):
        # Definir los límites superior e inferior
        lower_bound = self.Q1 - self.lambd * self.IQR
        upper_bound = self.Q3 + self.lambd * self.IQR
        # Filtrar los valores que están dentro de los límites
        return X[(X >= lower_bound) & (X <= upper_bound)]

    def set_output(self, transform='default'):
        # No modificar esta función
        return self

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definicion de las variables que pasarán por cada pipeline
numerical_columns = ['Price', 'Quantity']
categorical_columns = ['Invoice', 'StockCode', 'Description', 'InvoiceDate', 'Country', 'Customer ID']
lambd = 1.5
# Definicion del pipeline para las columnas numéricas usando IQR
numeric_transformations = Pipeline(steps=[
    ('iqr_outlier_removal', IQR(lambd))
])
# ColumnTransformer que aplica las transformaciones
column_transformer = ColumnTransformer([
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', 'passthrough', categorical_columns)
], verbose_feature_names_out=False)

# Aplicar las transformaciones
df_iqr = column_transformer.fit_transform(df)

# Convertir el resultado en DataFrame para facilitar su visualización
df_iqr = pd.DataFrame(df_iqr, columns=numerical_columns + categorical_columns)

# Eliminamos filas con valores nulos en las variables numéricas
df_iqr = df_iqr.dropna(subset=numerical_columns)


# Usamos explore_data en df_retail y en df_iqr
print("Distribución de variables antes de aplicar IQR:")
explore_data(df)
print("\nDistribución de variables después de aplicar IQR:")
explore_data(df_iqr)

*Reporte los cambios observados aquí.*

Observando los gráficos obtenidos para el precio, se obtiene que antes de la transformación la distribución del precio estaba fuertemente sesgada debido a la presencia de outliers con precios extremadamente altos, lo que dificultaba la visualización de los precios más comunes que se concentraban en el rango bajo. Después de la transformación, la eliminación de estos outliers y el escalado ajustaron la distribución de los valores, permitiendo una visualización más clara y precisa de los precios.

Por otra parte, al observar los gráficos obtenidos para la cantidad, se tiene que antes de la transformación se observaba una distribución altamente sesgada, con valores concentrados en un rango menor y la presencia de outliers que alcanzaban hasta 18,000. Después de la transformación, la eliminación de estos valores extremos hizo que la distribución se centrara en un rango mucho más razonable, entre 0 y 20, facilitando el análisis de las cantidades más comunes. 

Si se aumenta el valor de lambda en la transformación IQR, los límites para identificar outliers se amplían, lo que permite que más valores, incluso aquellos que antes serían considerados extremos, sean incluidos en el conjunto de datos. Esto reduce la cantidad de outliers eliminados, resultando en distribuciones más amplias y menos concentradas, ya que los valores extremos seguirán presentes. Gráficamente, se observa que los valores tanto de precio como cantidad aumentan en los valores transformados, esto dado se que están incluyendo más datos. 

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
# 0. Se define lambda
lambd = 1.5
# 1. Pipeline para variables categóricas con imputación por moda
categoric_transformations = Pipeline(steps=[
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))
])

# 2. Agregar un paso de imputación por la media en el pipeline numérico
numeric_transformations = Pipeline(steps=[
    ('iqr_outlier_removal', IQR(lambd)),
    ('mean_imputer', SimpleImputer(strategy='mean'))
])

# 3. Crear un ColumnTransformer con ambos pipelines
column_transformer = ColumnTransformer([
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)

# Aplicar el transformador a los datos para crear df_mean_imputer
df_mean_imputer = pd.DataFrame(column_transformer.fit_transform(df),
                               columns=numerical_columns + categorical_columns)


# 4. Comparando resultados
print("Exploración de df_mean_imputer:")
explore_data(df_mean_imputer)
print("\nExploración de df_iqr:")
explore_data(df_iqr)

Al observar el gráfico del histograma de precios de df_mean_imputer se muestra una distribución de precios más dispersa, lo que sugiere la presencia precios inusuales. En cambio, el gráfico de df_iqr, tras aplicar la eliminación de outliers mediante el rango intercuartil (IQR), presenta una distribución más controlada y limpia, sin precios extremadamente altos. El histograma de cantidad para df_mean_imputer muestra una distribución más dispersa, con varios valores altos en diferentes niveles de cantidad, como alrededor de 1, 5, 10, y 25 unidades. Sin embargo, el histograma para df_iqr tiene una distribución mucho más controlada, pero manteniendo la misma distribución de los datos que la data anterior, las barras más anchas en el gráfico de df_iqr sugieren que los datos están menos dispersos después de la eliminación de outliers, mientras que las barras más estrechas en df_mean_imputer reflejan una mayor variabilidad en los datos.

In [ ]:
# 5. Cambiar el imputer de `numeric_transformations` por KNNImputer
numeric_transformations_knn = Pipeline([
    ('remove_outliers', IQR(lambd)), 
    ('knn_imputer', KNNImputer(n_neighbors=1))
])

column_transformer_knn = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformations_knn, numerical_columns),
        ('categorical', categoric_transformations, categorical_columns)
    ],
    verbose_feature_names_out=False
)

column_transformer_knn.set_output(transform='pandas')

df_knn_imputer = column_transformer_knn.fit_transform(df)

In [ ]:
# 6. Comparando resultados
print("Exploración de df_knn_imputer:")
explore_data(df_knn_imputer)
print("\nExploración de df_iqr:")
explore_data(df_iqr)

*Diferencias en los datos:* Observamos que la imputación con KNN tiende a mantener la forma de las distribuciones y mantiene la correlación entre las variables, mientras que la eliminación de outliers con IQR reduce la variabilidad en los datos, pero puede llevar a distribuciones más condensadas y con menos ruido. Notamos que ambos enfoques tienen ventajas dependiendo del contexto del análisis: KNN es preferible cuando los valores faltantes no son muy numerosos y se desea preservar la estructura general de los datos, mientras que IQR es útil para eliminar efectos no deseados de outliers extremos.

In [ ]:
# 7. Comparando resultados knn_imputer y mean_imputer
print("Exploración de df_knn_imputer:")
explore_data(df_knn_imputer)
print("\nExploración de df_mean_imputer:")
explore_data(df_mean_imputer)

Se observa que el método KNN es claramente superior a la imputación por la media en este caso, ya que proporciona una mejor preservación de la estructura de los datos, la variabilidad y las relaciones entre características. Por lo tanto, el método KNN sería el mejor.

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
def custom_features(dataframe_in):
    #Pasamos a datetime la variable InvoiceDate
    dataframe_in['InvoiceDate'] = pd.to_datetime(dataframe_in['InvoiceDate'])
    
    # Agrupamos por Customer ID y obtener los valores de cada definicion
    grouped = dataframe_in.groupby('Customer ID').agg(
        length=('InvoiceDate', lambda x: (x.max() - x.min()).days),
        recency=('InvoiceDate', lambda x: (dataframe_in['InvoiceDate'].max() - x.max()).days),
        frequency=('InvoiceDate', 'count'),
        monetary=('Price', 'sum')  # Cambiado a 'sum' en lugar de 'mean'
    ).reset_index()

    # Calculamos periodicidad
    df_sorted = dataframe_in.sort_values(['Customer ID', 'InvoiceDate'])
    df_sorted['TimeDiff'] = df_sorted.groupby('Customer ID')['InvoiceDate'].diff().dt.days
    
    periodicity = df_sorted.groupby('Customer ID')['TimeDiff'].std().reset_index()
    periodicity.columns = ['Customer ID', 'Periodicity']

    # Combinamos ambos resultados
    result = pd.merge(grouped, periodicity, on='Customer ID', how='left')
    
    # Reemplazamos nulos en periodicidad con 0, para clientes con una sola compra
    result['Periodicity'].fillna(0, inplace=True)
    
    return result

In [ ]:
df_lrmfp = custom_features(df)
df_lrmfp.head()

#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [ ]:
# Función para convertir los datos a un DataFrame de pandas
def to_dataframe(df):
    return pd.DataFrame(df, columns=numerical_columns + categorical_columns)

# Crear el nuevo pipeline que encapsula el ColumnTransformer (KNN escogido) y calcula las LRMFP
retail_pipeline = Pipeline([
    ('col_transformer', column_transformer_knn),  # Aplicar el column transformer
    ('to_dataframe', FunctionTransformer(to_dataframe, validate=False)),  # Convertir a DataFrame
    ('custom_features', FunctionTransformer(custom_features))  # Calcular LRMFP
])

In [ ]:
df_custom = retail_pipeline.fit_transform(df)

In [ ]:
# Modificamos explore_data para poder calcular las variables que se nos piden
def explore_data(dataframe_in):
    # Plotea histogramas para las nuevas variables
    fig_length = px.histogram(dataframe_in, x='length', title='Histograma de Length')
    fig_recency = px.histogram(dataframe_in, x='recency', title='Histograma de Recency')
    fig_frequency = px.histogram(dataframe_in, x='frequency', title='Histograma de Frequency')
    fig_monetary = px.histogram(dataframe_in, x='monetary', title='Histograma de Monetary')
    fig_periodicity = px.histogram(dataframe_in, x='Periodicity', title='Histograma de Periodicity')

    # Muestra los histogramas
    fig_length.show()
    fig_recency.show()
    fig_frequency.show()
    fig_monetary.show()
    fig_periodicity.show()

explore_data(df_custom)

**Comentarios de feautures:**
- Lenght: se puede observar una gran cantidad de datos concentrados en valores bajos (0-50), lo que representa más de 1000 observaciones. A partir de 200, los datos comienzan a aumentar nuevamente, con un incremento significativo entre 250 y 300. Lo que podría representar diferentes categorías o subgrupos dentro de los datos.
- Recency:  se observa una distribución sesgada hacia la derecha, con la mayoría de los datos concentrados en valores bajos de recencia (0-50), lo que indica que muchos clientes o eventos tienen una recencia baja. Hay un pequeño aumento en los valores cercanos a 300, lo que podría indicar un grupo de clientes con una recencia significativamente alta.
- Frecuency:  se observa una distribución extremadamente sesgada a la derecha. La mayoría de los valores de frecuencia están muy concentrados en el rango bajo (0-200), con una caída rápida a medida que aumenta la frecuencia. 
- Monetary: e observa una distribución fuertemente sesgada hacia la derecha. La mayoría de los datos están concentrados en valores bajos (entre 0 y 2000), con un gran peak en los primeros rangos, lo que indica que la mayoría de las observaciones tienen valores monetarios pequeños.
- Periodicity: se observa que la mayoría de las observaciones están concentradas en los primeros valores (0-20), lo que indica que la mayoría de los eventos o transacciones ocurren con una periodicidad baja. Esto sugiere que la mayoría de los eventos tienen ciclos cortos y repetitivos, y solo un pequeño grupo tiene una periodicidad más amplia.

**Insight**

Un insight para el negocio podría ser que los clientes tienen a realizar compras con una frecuencia relativamente baja pero, se tiene que el gasto de ellos puede ser moderado ya que oscila entre los 0 y 100. Sin embargo, se puede ver que existen clientes que realizan compras de manera más frecuente. Con lo anterior se podría diferenciar dos segmentos de clientes que tienen diferentes comportamientos, lo cual podría ser util para poder personalizar estrategias de marketing y ventas para cada segmente, con la idea de porder máximizar el beneficio para la empresa y también tal vez fidelizar en mayor medida a los clientes.

### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):
    
    def fit(self, X):
        # Calcular y almacenar el mínimo y el máximo de cada característica
        self.max_ = X.max()  
        self.min_ = X.min()
        return self

    def transform(self, X):
        # Realizar la transformación Min-Max utilizando la fórmula
        return (X - self.min_) / (self.max_ - self.min_)

    def set_output(self, transform='default'):
        # No modificar este método
        return self

#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [ ]:
# Agregamos el paso minmax al pipeline numeric_transformations
numeric_transformations = Pipeline(steps=[
    ('iqr_outlier_removal', IQR(lambd=1.5)),  # Remoción de outliers
    ('mean_imputer', SimpleImputer(strategy='mean')),  # Imputación por la media
    ('minmax', MinMax())  # Escalado Min-Max
])

# Creamos el ColumnTransformer actualizado
column_transformer_update = ColumnTransformer([
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)

# Fijamos el parámetro set_output a pandas para obtener un DataFrame
column_transformer_update.set_output(transform='pandas')

# Aplicamos el pipeline a los datos y generar df_minmax
df_minmax = column_transformer.fit_transform(online_retail_data)

# Usamos explore_data
explore_data(df_minmax)

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

La utilidad de crear pipelines es que permiten estructurar el proceso de transformación de datos de manera modular, esto facilita la reutilización de los conjuntos de datos, debido a que cada etapa del pipeline se define secuencialmente. Esta estructura modular también automatiza la preparación de datos, garantizando consistencia y evitando errores humanos al aplicar las transformaciones manualmente. Además, los pipelines aseguran que las transformaciones se realicen en el orden correcto, lo que mejora la eficiencia y trazabilidad del proceso, permitiendo a los equipos rastrear fácilmente los cambios y mantener la calidad de los datos. Finalmente, los pipelines facilitan los ajustes y la optimización, ya que cualquier cambio en las transformaciones se puede implementar sin reescribir el código, lo que también simplifica la optimización de hiperparámetros en flujos de trabajo de Machine Learning.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>